In [ ]:
from matplotlib import pyplot as plt
import matplotlib as mpl
plt.style.use("ggplot")
mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 14
from scipy.integrate import solve_ivp
from scipy.integrate._ivp.ivp import OdeResult
from scipy.optimize import curve_fit
import seaborn as sns
import numpy as np
import tqdm, os, glob
import itertools

from ars_247_2025_final_project.models import TRAM_Model
from ars_247_2025_final_project.tools import pulse_model
from ars_247_2025_final_project.analysis import analyze_single_var_axis, analyze_multiple_var_axis
from ars_247_2025_final_project.vis import visualize_single_var_axis, visualize_two_var_grid

In [2]:
OUTPUT_DIR = os.path.abspath("..")
OUTPUT_PNG_DIR = os.path.join(OUTPUT_DIR, "Final_Project_PNG", "Fig2_Log")
print(f"Saving files to {OUTPUT_DIR}...")
OUTPUT_NPZ_DIR = os.path.join(OUTPUT_DIR, "npz")
DESIRED_NPZ = os.path.join(OUTPUT_NPZ_DIR, "log_parameter_space_results.npz")
USE_EXISTING_NPZ = False
if os.path.isfile(DESIRED_NPZ) > 0:
    answer = ""
    while answer != 'y' and answer != 'n':
        answer = input(f"Detected {DESIRED_NPZ}. Would you like to use it as your data? (y/n)")
    if answer == 'y': USE_EXISTING_NPZ = True

Saving files to /Users/alexandresathler/Documents/BioE_Program/Bioengineering Courses/ARS_247_2025_Final_Project...


In [3]:
HOUR = 3600
MINUTE = 60
# transcriptional pulse
transcriptional_pulse = {
    'peak_time': 1.5 * HOUR,
    'peak_value': 1,
    'width': 0.5 * HOUR
}

# phosphorylation pulse
phosphorylation_pulse = {
    'peak_time': 1.5 * HOUR // 4,
    'peak_value': 1,
    'width': 0.5 * HOUR // 4 # 45 mins peak phosphorylation pulse
}

# optogenetic pulse
optogenetic_pulse = {
    'peak_time': 1.5 * HOUR // 10, # peak at 9 mins
    'peak_value': 1,
    'width': 0.5 * HOUR // 10 # 3 mins peak activity of pulse
}

og_system_params = {
    'PROM': 1,
    'V_A': 0.47, #um/s - mean for dynein / kinesin
    'R_M': 2.5, #um - mean for mammalian cells is 2.5; best results w/ 20
    'K_TEV': 0.015, #/s - physiological is ~0.15
    'K_TVMV': 0.015, #/s - physiological is ~0.15
    'K_TRAM_DEG': 0.00001, #/s - got interesting results with this being 0.01; physiological is ~0.00001
    'K_TF_DEG': 0.01 #/s - got interesting results with this being 0.1; physiological is ~0.01 w/ fast degron
}

system_params = og_system_params.copy()

## Figure 2

In [4]:
# Creating the linear parameter space for each parameter of interest
linear_parameter_space = np.concat([1 / np.logspace(1, 3, 3)[::-1], np.ndarray([1]), np.logspace(1, 3, 3)]).astype(np.float32)
print(linear_parameter_space)
speed_ranges = linear_parameter_space * og_system_params['V_A']
protease_ranges = linear_parameter_space * og_system_params['K_TEV']
deg_tram_ranges = linear_parameter_space * og_system_params['K_TRAM_DEG']
deg_tf_ranges = linear_parameter_space * og_system_params['K_TF_DEG']

[1.e-03 1.e-02 1.e-01 1.e+00 1.e+01 1.e+02 1.e+03]


In [5]:
24 * HOUR

86400

In [ ]:
if not USE_EXISTING_NPZ:
    # Combinatorially constructing all possible parameter combinations based on linear parameter space
    all_combinations = np.array(list(itertools.product(speed_ranges, protease_ranges, deg_tram_ranges, deg_tf_ranges)))
    models_comb = []
    T = int(24 * HOUR)
    N_DOMAINS = 6
    solns_comb = np.zeros(shape=(len(all_combinations), 4 * N_DOMAINS + 1, T))
    system_params = og_system_params.copy()
    for idx, (speedx, protx, degrx, degtx) in enumerate(tqdm.tqdm(all_combinations)):
        system_params['V_A'] = speedx
        system_params['K_TEV'], system_params['K_TVMV'] = protx, protx
        system_params['K_TRAM_DEG'], system_params['K_TF_DEG'] = degrx, degtx

        this_model = TRAM_Model(
            n_delays=N_DOMAINS,
            pulse='gaussian',
            pulse_params=optogenetic_pulse,
            sys_config=system_params
        )

        models_comb.append(this_model)
        solns_comb[idx,...] = this_model.solve_delay_ivp(0, T)['y']

else:
    print("User elected to use existing NPZ as input data, skipping IVP solving step...")

 94%|█████████▍| 2261/2401 [40:24:41<19:35:17, 503.69s/it] 

Stacking results into a solution object and saving to NPZ file

In [ ]:
if not USE_EXISTING_NPZ: 
    npz_save_dict = {
        'V_A': speed_ranges,
        'K_TEV': protease_ranges,
        'K_TRAM_DEG': deg_tram_ranges,
        'K_TF_DEG': deg_tf_ranges,
        'n_ranges': 4,
        'combinations': all_combinations,
        'solutions': solns_comb
    }
    
    np.savez(
        file=DESIRED_NPZ,
        **npz_save_dict
    )
else:
    npz_save_dict = np.load(DESIRED_NPZ)

### Figure 2a

In [ ]:
# Getting pulse start time
p_model = pulse_model('gaussian', params=optogenetic_pulse)
p_peak_time = np.argmax(
    solve_ivp(
        fun = p_model,
        t_span = [0, 4 * 3600],
        y0 = [0],
        max_step = 1)
)

In [ ]:
key, key_range, delays, amplitudes = analyze_single_var_axis \
(
    input_data=npz_save_dict,
    key='V_A',
    default_params=og_system_params,
    start_time=p_peak_time
)

In [ ]:
visualize_single_var_axis(
    key = key,
    delays = delays,
    amplitudes = amplitudes,
    delay_y_axis_min = 0,
    delay_y_axis_max = 5500,
    delay_y_axis_step = HOUR / 4,
    divide_delay_y_axis_by = HOUR,
    delay_y_axis_label = "Time (hr)"
)
plt.savefig(os.path.join(OUTPUT_PNG_DIR, "Fig2a-V_A.png"))

In [ ]:
key, key_range, delays, amplitudes = analyze_single_var_axis \
(
    input_data=npz_save_dict,
    key='K_TEV',
    default_params=og_system_params,
    start_time=p_peak_time
)

In [ ]:
visualize_single_var_axis(
    key = key,
    delays = delays,
    amplitudes = amplitudes,
    delay_y_axis_min = 0,
    delay_y_axis_max = 5500,
    delay_y_axis_step = HOUR / 4,
    divide_delay_y_axis_by = HOUR,
    delay_y_axis_label = "Time (hr)"
)
plt.savefig(os.path.join(OUTPUT_PNG_DIR, "Fig2a-K_TEV.png"))

In [ ]:
key, key_range, delays, amplitudes = analyze_single_var_axis \
(
    input_data=npz_save_dict,
    key='K_TRAM_DEG',
    default_params=og_system_params,
    start_time=p_peak_time
)

In [ ]:
visualize_single_var_axis(
    key = key,
    delays = delays,
    amplitudes = amplitudes,
    delay_y_axis_min = 0,
    delay_y_axis_max = 5500,
    delay_y_axis_step = HOUR / 4,
    divide_delay_y_axis_by = HOUR,
    delay_y_axis_label = "Time (hr)"
)
plt.savefig(os.path.join(OUTPUT_PNG_DIR, "Fig2a-K_TRAM_DEG.png"))

In [ ]:
key, key_range, delays, amplitudes = analyze_single_var_axis \
(
    input_data=npz_save_dict,
    key='K_TF_DEG',
    default_params=og_system_params,
    start_time=p_peak_time
)

In [ ]:
visualize_single_var_axis(
    key = key,
    delays = delays,
    amplitudes = amplitudes,
    delay_y_axis_min = 0,
    delay_y_axis_max = 5500,
    delay_y_axis_step = HOUR / 4,
    divide_delay_y_axis_by = HOUR,
    delay_y_axis_label = "Time (hr)"
)
plt.savefig(os.path.join(OUTPUT_PNG_DIR, "Fig2a-K_TF_DEG.png"))

### Multivariate analysis

In [ ]:
import matplotlib.colors as mcolors

In [ ]:
amp_colors = ['black', 'green']
amp_cmap = mcolors.LinearSegmentedColormap.from_list('amp', amp_colors)

In [ ]:
keys = ["K_TF_DEG", "K_TEV"]
keys, key_ranges, delays, amplitudes = analyze_multiple_var_axis(
    input_data=npz_save_dict,
    keys=keys,
    default_params=og_system_params,
    start_time=p_peak_time
)

In [ ]:
visualize_two_var_grid(
    data=amplitudes,
    keys=keys,
    plt_title="",
    cbar_step=50,
    cbar_title="Expression (REU)",
    # imshow_cmap="magma"
    imshow_cmap = amp_cmap
)
plt.savefig(os.path.join(OUTPUT_PNG_DIR, "Fig2b-Amp.png"))

In [ ]:
del_colors = ['black', 'm']
del_cmap = mcolors.LinearSegmentedColormap.from_list('del', del_colors)

In [ ]:
keys = ["V_A", "K_TEV"]
keys, key_ranges, delays, amplitudes = analyze_multiple_var_axis(
    input_data=npz_save_dict,
    keys=keys,
    default_params=og_system_params,
    start_time=p_peak_time
)

In [ ]:
visualize_two_var_grid(
    data=delays,
    keys=keys,
    plt_title="",
    cbar_step=HOUR / 4, # 60 second step for time minutes
    cbar_title="Time (hrs)",
    divide_cbar_label_by=HOUR,
    # imshow_cmap="magma"
    imshow_cmap=del_cmap
)
plt.savefig(os.path.join(OUTPUT_PNG_DIR, "Fig2b-Delay.png"))